# chapter 4. Classification

## Gradient Boosting Machine

### GBM의 개요 및 실습

**부스팅 알고리즘**은 **여러 개의 약한 학습기를 순차적으로 학습-예측하면서 잘못 예측한 데이터에 가중치 부여를 통해 오류를 개선해나가면서 학습하는 방식**이다. <br>
부스팅의 대표적인 구현은 **AdaBoost(Adaptive Boosting)** 와 **그래디언트 부스트**가 있다. <br>
**에이다 부스트**는 오류 데이터에 가중치를 부여하면서 부스팅을 수행하는 대표적인 알고리즘이다. <br>

**GBM**은 CART 기반의 다른 알고리즘과 마찬가지로 **분류, 회귀가 가능**하다. <br>
사이킷런은 **GBM 기반의 분류를 위해 GradientBoostingClassifier 클래스를 제공**한다. 

In [1]:
# 이전 Random Forest Data

import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

# feature.txt 파일에는 피처 이름 index와 피처명이 공백으로 분리되어 있다. -> DataFrame으로 Load
feature_name_df = pd.read_csv('/Users/1001l1000/Documents/AI/Jen/data/human_activity/features.txt', sep = '\s+',
                             header = None, names = ['column_index', 'column_name'])

# 피처명 index를 제거하고, 피처명만 리스트 객체로 생성한 뒤 샘플로 10개만 추출
feature_name = feature_name_df.iloc[:, 1].values.tolist()
print('전체 피처명에서 10개만 추출 : ', feature_name[:10])

def get_new_feature_name_df(old_feature_name_df):
    feature_dup_df = pd.DataFrame(data = old_feature_name_df.groupby('column_name').cumcount(),
                                 columns = ['dup_cnt'])
    feature_dup_df = feature_dup_df.reset_index()
    new_feature_name_df = pd.merge(old_feature_name_df.reset_index(), feature_dup_df, how = 'outer')
    new_feature_name_df['column_name'] = new_feature_name_df[['column_name', 
                                                              'dup_cnt']].apply(lambda x : x[0]+'_'+str(x[1])
                                                                               if x[1] > 0 else x[0], axis = 1)
    new_feature_name_df = new_feature_name_df.drop(['index'], axis = 1)
    return new_feature_name_df

def get_human_dataset( ): 
    # 각 데이터 파일들은 공백으로 분리되어 있으므로 read_csv에서 공백 문자를 sep으로 할당.
    feature_name_df = pd.read_csv('/Users/1001l1000/Documents/AI/Jen/data/human_activity/features.txt',sep = '\s+',
                        header = None, names = ['column_index','column_name'])
    
    # 중복된 피처명을 수정하는 get_new_feature_name_df()를 이용, 신규 피처명 DataFrame생성. 
    new_feature_name_df = get_new_feature_name_df(feature_name_df)
    
    # DataFrame에 피처명을 컬럼으로 부여하기 위해 리스트 객체로 다시 변환
    feature_name = new_feature_name_df.iloc[:, 1].values.tolist()
    
    # 학습 피처 데이터 셋과 테스트 피처 데이터을 DataFrame으로 로딩. 컬럼명은 feature_name 적용
    X_train = pd.read_csv('/Users/1001l1000/Documents/AI/Jen/data/human_activity/train/X_train.txt', sep = '\s+', names = feature_name )
    X_test = pd.read_csv('/Users/1001l1000/Documents/AI/Jen/data/human_activity/test/X_test.txt', sep = '\s+', names = feature_name)
    
    # 학습 레이블과 테스트 레이블 데이터을 DataFrame으로 로딩하고 컬럼명은 action으로 부여
    y_train = pd.read_csv('/Users/1001l1000/Documents/AI/Jen/data/human_activity/train/y_train.txt', sep = '\s+',header = None, names = ['action'])
    y_test = pd.read_csv('/Users/1001l1000/Documents/AI/Jen/data/human_activity/test/y_test.txt', sep = '\s+',header = None, names = ['action'])
    
    # 로드된 학습 / 테스트용 DataFrame을 모두 반환 
    return X_train, X_test, y_train, y_test

X_train, X_test, y_train, y_test = get_human_dataset()

전체 피처명에서 10개만 추출 :  ['tBodyAcc-mean()-X', 'tBodyAcc-mean()-Y', 'tBodyAcc-mean()-Z', 'tBodyAcc-std()-X', 'tBodyAcc-std()-Y', 'tBodyAcc-std()-Z', 'tBodyAcc-mad()-X', 'tBodyAcc-mad()-Y', 'tBodyAcc-mad()-Z', 'tBodyAcc-max()-X']


In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
import time
import warnings
warnings.filterwarnings('ignore')

X_train, X_test, Y_train, Y_test = get_human_dataset()

# GBM 수행 시간 측정을 위한 것이다. 시작 시간 설정
start_time = time.time()

gb_clf = GradientBoostingClassifier(random_state = 0)
gb_clf.fit(X_train, Y_train)
gb_pred = gb_clf.predict(X_test)
gb_accuracy = accuracy_score(Y_test, gb_pred)

print('GBM 정확도 : {0:.4f}'.format(gb_accuracy))
print('GBM 수행 시간 : {0:.1f} 초'.format(time.time() - start_time))

특히 수행 시간 문제는 GBM이 해결해야 할 문제이다. <br>
사이킷런의 GradientBoostingClassifier는 약한 학습기의 순차적인 예측 오류 보정을 통해 학습을 수행하므로 멀티 CPU 코어 시스템을 사용하더라도 병렬 처리가 지원되지 않아서 대용량 데이터의 경우 학습에 매우 많은 시간이 필요하다. 

### GBM 하이퍼 파라미터 

- loss : 경사 하강법에서 사용할 비용 함수를 지정한다. 특별한 이유가 없으면 기본값인 deviance를 그대로 적용한다. <br>
- learning_rate : GBM이 학습을 진행할 때마다 적용하는 학습률이다. <br>
- n_estimators : weak learner의 갯수이다. <br>
- subsample : weak learner가 학습에 사용하는 데이터의 샘플링 비율이다. 기본값은 1이고 이는 전체 학습 데이터를 기반으로 학습한다는 의미이다. 